I plan to update the model in a timely manner. 

* This is probably the maximum in LB, as some are already in LB with a score of 5.434. 
* Therefore, from here on, it is necessary to suppress overfitting and increase the Private Score, rather than aiming at the value of LB.
https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/discussion/318015

**I had the above idea, but I realized in the discussion that the idea of portfolio is important, so I corrected it.**
https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/discussion/318055

**Rank after considering the target and risk with reference to the following.Please comment if you don't understand**
https://www.kaggle.com/code/tensorchoko/jpx-eda-model-jp-en

# **if it is useful for you, please vote!**

In [1]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [3]:
prices["Date"]

0         2021-12-06
1         2021-12-06
2         2021-12-06
3         2021-12-06
4         2021-12-06
             ...    
111995    2022-02-28
111996    2022-02-28
111997    2022-02-28
111998    2022-02-28
111999    2022-02-28
Name: Date, Length: 112000, dtype: object

In [4]:
NDAYS = 50
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [5]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [6]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","Avg","High","Open","Close","Low","Volume"]]
y=prices[["Target"]]
codes = X.SecuritiesCode.unique()

In [7]:
import optuna

def objectives(trial):
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 300, 4000),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

In [8]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objectives, n_trials=20)

# 最適パラメータ取得
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
# 最適パラメータで学習/予測    
model_o = LGBMRegressor(**params_best)#

[I 2022-04-11 08:12:13,416] A new study created in memory with name: no-name-b83ef384-632b-4b3b-ab98-61bb229fad99
[I 2022-04-11 08:13:32,046] Trial 0 finished with value: 0.9999316269550744 and parameters: {'num_leaves': 2331, 'n_estimators': 718, 'max_bin': 61, 'learning_rate': 0.5448831829968969}. Best is trial 0 with value: 0.9999316269550744.
[I 2022-04-11 08:14:25,944] Trial 1 finished with value: 0.98344274050321 and parameters: {'num_leaves': 1867, 'n_estimators': 650, 'max_bin': 45, 'learning_rate': 0.8917730007820798}. Best is trial 0 with value: 0.9999316269550744.
[I 2022-04-11 08:15:39,790] Trial 2 finished with value: 0.9999281022959217 and parameters: {'num_leaves': 3866, 'n_estimators': 389, 'max_bin': 80, 'learning_rate': 0.5288949197529045}. Best is trial 0 with value: 0.9999316269550744.
[I 2022-04-11 08:17:20,734] Trial 3 finished with value: 0.24909089862614076 and parameters: {'num_leaves': 2402, 'n_estimators': 927, 'max_bin': 9, 'learning_rate': 0.087129299701540

In [9]:
model_o.fit(X,y)

LGBMRegressor(learning_rate=0.6818202991034834, max_bin=95, n_estimators=621,
              num_leaves=2565, random_seed=0)

In [10]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    df = sample_prediction[["Date","SecuritiesCode","Avg"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df["Volume"] = prices["Volume"]
    df.Date = pd.to_datetime(df.Date)
    sample_prediction["Volume"] = df["Volume"]
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction["rate"] = sample_prediction["Prediction"]/(np.log(sample_prediction["Volume"]+1))
    sample_prediction = sample_prediction.sort_values(by = "rate", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
